In [1]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install bitsandbytes

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-6dnp99zi
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-6dnp99zi
  Resolved https://github.com/huggingface/transformers.git to commit 05260a1fc1c8571a2b421ce72b680d5f1bc3e5a4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10290584 sha256=b81d3f85cb8c6fdbd8f5071d5dcd8ae78585a66c8256936e122d5e15116f811b
  Stored in directory: /tmp/pip-ephem-wheel-cache-kmb6bnyy/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
  Cloning https://github.co

In [2]:
# importing the packages
import torch
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

warnings.filterwarnings('ignore')

In [3]:
# loading the model from huggingface
model = AutoModelForCausalLM.from_pretrained(
    'NousResearch/Llama-2-7b-chat-hf',
    load_in_4bit=True,                                                            # loading the model with 4 bit precision, so as to show the example on Colab GPU
    device_map='auto'
)
tokenizer = AutoTokenizer.from_pretrained('NousResearch/Llama-2-7b-chat-hf')
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [4]:
model.config

LlamaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "NousResearch/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": tru

In [5]:
# setting the model generation configuration
generation_config = GenerationConfig(
    do_sample = True,
    temperature = 0.8,
    repetition_penalty = 1.5,
    max_new_tokens = 256
)

# Zero Shot

In [6]:
inputs = tokenizer("I'll bet the video game is alot more fun than the film.\Sentiment:", return_tensors="pt")
print(inputs)
input_ids = inputs["input_ids"].to('cuda')

{'input_ids': tensor([[    1,   306, 29915,   645,  1010,   278,  4863,  3748,   338,   394,
           327,   901,  2090,  1135,   278,  2706,  7790, 29903,   296,  2073,
         29901]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [7]:
# passing the prompt tokens to the generative model to generate the response tokens
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config)

In [8]:
# decode the model generated tokens, and printing the generated output.
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

I'll bet the video game is alot more fun than the film.\Sentiment:_PositiveRating:3
 Hinweis: The tone of this sentence could be interpreted as either positive or negative, depending on how it is phrased and what context in which it appears. As written above. It would appear to express a preference for playing games over watching movies (a common activity) but also suggests that some people might find films exciting too! Maybe there are certain genres/themes within gaming where they take precedence? For example,"adventure"vs."action-packed". Or perhaps sequential storytelling through levels offers something distinct from narrative arcs found inside cinematic productions; these options offer different ways at crafting engaging experiences without needlessly relying solely upon visuals alone(i e-, CGI).In any case.,the statement remains open ended allowing readers make their own assumptions based off individual perspectives


# Few Shot

In [10]:
inputs = tokenizer('''
Text: (lawrence bounces) all over the stage, dancing, running, sweating, mopping his face and generally displaying the wacky talent that brought him fame in the first place.
Sentiment: positive

Text: despite all evidence to the contrary, this clunker has somehow managed to pose as an actual feature movie, the kind that charges full admission and gets hyped on
tv and purports to amuse small children and ostensible adults.
Sentiment: negative

Text: for the first time in years, de niro digs deep emotionally, perhaps because he's been stirred by the powerful work of his co-stars.
Sentiment: positive

Text: i'll bet the video game is a lot more fun than the film.
\Sentiment:''', return_tensors="pt")
input_ids = inputs["input_ids"].to('cuda')

In [11]:
# passing the prompt tokens to the generative model to generate the response tokens
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config)

In [12]:
# decode the model generated tokens, and printing the generated output.
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

Text: (lawrence bounces) all over the stage, dancing, running, sweating, mopping his face and generally displaying the wacky talent that brought him fame in the first place.
Sentiment: positive

Text: despite all evidence to the contrary, this clunker has somehow managed to pose as an actual feature movie, the kind that charges full admission and gets hyped on
tv and purports to amuse small children and ostensible adults.
Sentiment: negative

Text: for the first time in years, de niro digs deep emotionally, perhaps because he's been stirred by the powerful work of his co-stars.
Sentiment: positive

Text: i'll bet the video game is a lot more fun than the film.
\Sentiment: mixed


# Instruct Prompting

In [13]:
inputs = tokenizer('''Please label the sentiment towards the movie of the given movie review. The sentiment label should be "positive" or "negative".
Text: i'll bet the video game is a lot more fun than the film.
\Sentiment:''', return_tensors="pt")
input_ids = inputs["input_ids"].to('cuda')

In [14]:
# passing the prompt tokens to the generative model to generate the response tokens
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config)

In [15]:
# decode the model generated tokens, and printing the generated output.
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

Please label the sentiment towards the movie of the given movie review. The sentiment label should be "positive" or "negative".
Text: i'll bet the video game is a lot more fun than the film.
\Sentiment: Positive
